In [ ]:
# ✅ Set your API key here
import os
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # replace with your key

import json
import logging
from typing import Dict
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import AIMessage, HumanMessage

# ✅ Import functions from your notebook file (make sure LLMs_test.py exists in same dir)
from LLMs_test import generate_question, evaluate_answer

C:\Users\rimsha\anaconda3\envs\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


# -------------------------------------------------------
# 🧠 1. Question Generation Chain
# -------------------------------------------------------
def get_question_generation_chain():
    """
    Creates a chain that generates interview questions using the latest ChatOpenAI.
    """
    template = """You are an AI interviewer generating {round_type} interview questions.
Candidate context: {context}
Generate one clear and concise question. Return only the question text."""

    prompt = PromptTemplate(
        input_variables=["round_type", "context"],
        template=template
    )

    # ✅ Use ChatOpenAI (new API)
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

    # ✅ Use new pipe syntax
    chain = prompt | llm
    return chain


# -------------------------------------------------------
# 🧩 2. Answer Evaluation Chain
# -------------------------------------------------------
def get_answer_evaluation_chain():
    """
    Wraps your evaluate_answer() function into a LangChain-like callable.
    """

    def _run(question: str, candidate_answer: str) -> Dict:
        # Directly call your notebook function
        result = evaluate_answer(question, candidate_answer)
        return result

    # ✅ Simple chain wrapper
    class EvalChain:
        def __init__(self):
            self.memory = ConversationBufferMemory(memory_key="history", return_messages=True)

        def __call__(self, question: str, candidate_answer: str):
            result = _run(question, candidate_answer)
            self.memory.chat_memory.add_user_message(candidate_answer)
            self.memory.chat_memory.add_ai_message(result["evaluation"]["feedback"])
            return result

    return EvalChain()


# -------------------------------------------------------
# 💬 3. Example Usage (Testing)
# -------------------------------------------------------
if __name__ == "__main__":
    print("🚀 Step 3 — LangChain Integration Demo (Latest API Version)\n")

    # 1️⃣ Create chains
    q_chain = get_question_generation_chain()
    eval_chain = get_answer_evaluation_chain()

    # 2️⃣ Generate a question
    inputs = {"round_type": "HR", "context": "Candidate is a software engineer with 2 years of experience"}
    question_output = q_chain.invoke(inputs)
    generated_question = question_output.content  # get text output from ChatMessage

    print("\n🧠 Generated Question:")
    print(generated_question)

    # 3️⃣ Candidate answer
    candidate_ans = "I once resolved a conflict by organizing a team meeting and discussing responsibilities openly."

    # 4️⃣ Evaluate answer
    evaluation_result = eval_chain(generated_question, candidate_ans)
    print("\n✅ Evaluation Result:")
    print(json.dumps(evaluation_result, indent=2))

🚀 Step 3 — LangChain Integration Demo (Latest API Version)



C:\Users\rimsha\AppData\Local\Temp\ipykernel_13756\948842048.py:22: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
C:\Users\rimsha\AppData\Local\Temp\ipykernel_13756\948842048.py:45: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(memory_key="history", return_messages=True)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:LLMs_test:Loading embedding model 'all-MiniLM-L6-v2' (this may download files)...
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: 


🧠 Generated Question:
Can you describe a challenging project you worked on and how you overcame the obstacles you faced?


INFO:LLMs_test:Computing embeddings for canonical answers...
Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.76it/s]
ERROR:LLMs_test:OpenAI evaluation failed: 

You tried to access openai.Model, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
Traceback (most recent call last):
  File "C:\Users\rimsha\OneDrive\Desktop\AI-Course\Project-AI-Powered Job Interview Coach\notebooks\LLMs_test.py", line 212, in _llm_evaluate_openai
    model="gpt-4o" if "gpt-4o" in (openai.Model.list().data[0].id if hasattr(openai,'Model') else "") else "gpt-4o-mini",
              


✅ Evaluation Result:
{
  "question": "Can you describe a challenging project you worked on and how you overcame the obstacles you faced?",
  "candidate_answer": "I once resolved a conflict by organizing a team meeting and discussing responsibilities openly.",
  "matched_canonical": {
    "id": "hr_01",
    "answer": "I used the STAR method: Situation - our team missed a deadline; Task - I coordinated communication; Action - scheduled a meeting, redistributed tasks, negotiated new timelines; Result - we delivered with acceptable quality and learned to align better."
  },
  "similarity": 0.4850238550784322,
  "evaluation": {
    "relevance": 0.4850238550784322,
    "completeness": 0.43782298276150583,
    "clarity": 1.0,
    "feedback": "Similarity to canonical answer: 0.49. Expand more on examples and outcomes.",
    "suggestions": "Use STAR structure for behavioral answers; add specific examples and metrics.",
    "matched_canonical_id": "hr_01",
    "similarity": 0.4850238550784322
 